In [192]:
import os
import pandas as pd
pd.options.display.max_rows = None
import matplotlib.pyplot as plt
import numpy as np
import datetime
import time
from datetime import timedelta
cantonKeys = ['AG','AI','AR', 'BE', 'BL', 'BS', 'FR', 'GE', 'GL', 'GR', 'JU', 'LU', 'NE', 'NW', 'OW', 'SG', 'SH', 'SO', 'SZ', 'TG', 'TI', 'UR', 'VD', 'VS', 'ZG','ZH']


dict = {}
for cantonId in cantonKeys:
    dict[cantonId] = pd.read_csv("data/dailyFeatures/"+cantonId+".csv").set_index('date')
    dict[cantonId].index = pd.to_datetime(dict[cantonId].index)




In [193]:
# generate date ranges
dataStart = pd.Timestamp('2020-02-15')
dataEnd = pd.Timestamp('2021-04-05')

daysIn = 14 # last n days of input
daysOut = 7 # next n days of output

listOfInputIntervals = []
listOfOutputIntervals = []
for e in pd.date_range(start=dataStart,end=dataEnd, freq='D'):
    if (e+timedelta(days = daysIn+daysOut-1) <= dataEnd.date()):
        listOfInputIntervals.append((e.date(),(e+timedelta(days = (daysIn-1))).date()))
        listOfOutputIntervals.append(((e+timedelta(days = daysIn)).date(),(e+timedelta(days = daysIn+daysOut-1)).date()))
    
display(listOfInputIntervals)
display(listOfOutputIntervals)

[(datetime.date(2020, 2, 15), datetime.date(2020, 2, 28)),
 (datetime.date(2020, 2, 16), datetime.date(2020, 2, 29)),
 (datetime.date(2020, 2, 17), datetime.date(2020, 3, 1)),
 (datetime.date(2020, 2, 18), datetime.date(2020, 3, 2)),
 (datetime.date(2020, 2, 19), datetime.date(2020, 3, 3)),
 (datetime.date(2020, 2, 20), datetime.date(2020, 3, 4)),
 (datetime.date(2020, 2, 21), datetime.date(2020, 3, 5)),
 (datetime.date(2020, 2, 22), datetime.date(2020, 3, 6)),
 (datetime.date(2020, 2, 23), datetime.date(2020, 3, 7)),
 (datetime.date(2020, 2, 24), datetime.date(2020, 3, 8)),
 (datetime.date(2020, 2, 25), datetime.date(2020, 3, 9)),
 (datetime.date(2020, 2, 26), datetime.date(2020, 3, 10)),
 (datetime.date(2020, 2, 27), datetime.date(2020, 3, 11)),
 (datetime.date(2020, 2, 28), datetime.date(2020, 3, 12)),
 (datetime.date(2020, 2, 29), datetime.date(2020, 3, 13)),
 (datetime.date(2020, 3, 1), datetime.date(2020, 3, 14)),
 (datetime.date(2020, 3, 2), datetime.date(2020, 3, 15)),
 (dateti

[(datetime.date(2020, 2, 29), datetime.date(2020, 3, 6)),
 (datetime.date(2020, 3, 1), datetime.date(2020, 3, 7)),
 (datetime.date(2020, 3, 2), datetime.date(2020, 3, 8)),
 (datetime.date(2020, 3, 3), datetime.date(2020, 3, 9)),
 (datetime.date(2020, 3, 4), datetime.date(2020, 3, 10)),
 (datetime.date(2020, 3, 5), datetime.date(2020, 3, 11)),
 (datetime.date(2020, 3, 6), datetime.date(2020, 3, 12)),
 (datetime.date(2020, 3, 7), datetime.date(2020, 3, 13)),
 (datetime.date(2020, 3, 8), datetime.date(2020, 3, 14)),
 (datetime.date(2020, 3, 9), datetime.date(2020, 3, 15)),
 (datetime.date(2020, 3, 10), datetime.date(2020, 3, 16)),
 (datetime.date(2020, 3, 11), datetime.date(2020, 3, 17)),
 (datetime.date(2020, 3, 12), datetime.date(2020, 3, 18)),
 (datetime.date(2020, 3, 13), datetime.date(2020, 3, 19)),
 (datetime.date(2020, 3, 14), datetime.date(2020, 3, 20)),
 (datetime.date(2020, 3, 15), datetime.date(2020, 3, 21)),
 (datetime.date(2020, 3, 16), datetime.date(2020, 3, 22)),
 (datetime

In [194]:
######### constructing the inputs #########
for cantonId in cantonKeys:
    features = pd.DataFrame()
    # construction of input features
    for t in listOfInputIntervals:
        # features which will be averaged over the whole input interval
        averageFeatures = ['Cases inz_entries 0 - 9','Cases inz_entries 10 - 19','Cases inz_entries 20 - 29',
                  'Cases inz_entries 30 - 39','Cases inz_entries 40 - 49','Cases inz_entries 50 - 59',
                  'Cases inz_entries 60 - 69','Cases inz_entries 70 - 79','Cases inz_entries 80+',
                  'Cases inzsumTotal 0 - 9','Cases inzsumTotal 10 - 19','Cases inzsumTotal 20 - 29',
                  'Cases inzsumTotal 30 - 39','Cases inzsumTotal 40 - 49','Cases inzsumTotal 50 - 59',
                  'Cases inzsumTotal 60 - 69','Cases inzsumTotal 70 - 79','Cases inzsumTotal 80+',
                  'Death inz_entries 0 - 9','Death inz_entries 10 - 19','Death inz_entries 20 - 29',
                  'Death inz_entries 30 - 39','Death inz_entries 40 - 49','Death inz_entries 50 - 59',
                  'Death inz_entries 60 - 69','Death inz_entries 70 - 79','Death inz_entries 80+',
                  'Death inzsumTotal 0 - 9','Death inzsumTotal 10 - 19','Death inzsumTotal 20 - 29',
                  'Death inzsumTotal 30 - 39','Death inzsumTotal 40 - 49','Death inzsumTotal 50 - 59',
                  'Death inzsumTotal 60 - 69','Death inzsumTotal 70 - 79','Death inzsumTotal 80+',
                  'Hosp inz_entries 0 - 9','Hosp inz_entries 10 - 19','Hosp inz_entries 20 - 29',
                  'Hosp inz_entries 30 - 39','Hosp inz_entries 40 - 49','Hosp inz_entries 50 - 59',
                  'Hosp inz_entries 60 - 69','Hosp inz_entries 70 - 79','Hosp inz_entries 80+',
                  'Hosp inzsumTotal 0 - 9','Hosp inzsumTotal 10 - 19','Hosp inzsumTotal 20 - 29',
                  'Hosp inzsumTotal 30 - 39','Hosp inzsumTotal 40 - 49','Hosp inzsumTotal 50 - 59',
                  'Hosp inzsumTotal 60 - 69','Hosp inzsumTotal 70 - 79','Hosp inzsumTotal 80+',
                  'Cases inz_entries female','Cases inz_entries male','Cases inzsumTotal female',
                  'Cases inzsumTotal male','Death inz_entries female','Death inz_entries male',
                  'Death inzsumTotal female','Death inzsumTotal male','Hosp inz_entries female',
                  'Hosp inz_entries male','Hosp inzsumTotal female','Hosp inzsumTotal male', 
                  'VaccDosesAdministered per100PersonsTotal',
                  'FullyVaccPersons per100PersonsTotal',
                  'anteil_pos',
                  'variant_error',
                  'case_inzsumTotal','hosp_inzsumTotal','death_inzsumTotal','test_inzsumTotal','case_inz_entries',
                  'hosp_inz_entries','death_inz_entries','test_inz_entries','testPositvity',
                  'median_R_mean','R_error',
                  'meanNeighborIncidence','maxNeighborIncidence',
                  'kofStrigency',
                  'Borders','Events','Gatherings/private events','Demonstrations',
                  'Primary (includes kindergarten) and lower secondary school','Upper secondary school, vocational schools and higher education',
                  'universities and other educational establishments\xa0','Mountain railways','Homeworking','Restaurants',
                  'Discos/Nightclubs','Shops/Markets','Penalties','Cultural, entertainment and recreational facilities',
                  'Sport/Wellness facilities','Sport activities','Religious services','Singing allowed','maskMandatories',
                  'ICU_AllPatients_inz','ICU_Covid19Patients_inz','ICU_Capacity_inz','Total_AllPatients_inz',
                  'Total_Covid19Patients_inz','Total_Capacity_inz','ICU_NonCovid19Patients_inz','ICU_FreeCapacity_inz',
                  'Total_NonCovid19Patients_inz','Total_FreeCapacity_inz'
                 ]

        # features which will be direct input for every day of the input interval
        # attention: this can potentially increase the number of input features significantly
        # added features are len(directFea)*daysIn
        # only add features for which have a large variance from one day to another day
        directFeatures = ['retail_and_recreation_percent_change_from_baseline',
                     'grocery_and_pharmacy_percent_change_from_baseline',
                     'parks_percent_change_from_baseline',
                     'transit_stations_percent_change_from_baseline',
                     'workplaces_percent_change_from_baseline',
                     'residential_percent_change_from_baseline',
                     'intervistaMob',
                     'isHolyday',
                     'temp_min','temp_max','clouds','precipitation']

        # average features
        featureRow = dict[cantonId][t[0]:t[1]][averageFeatures].mean().to_frame().transpose()

        # direct features
        for f in directFeatures:
            directFe = dict[cantonId][t[0]:t[1]][[f]].transpose()
            directFe.columns = [f+'_day_'+str(d) for d in range(0,daysIn)]
            directFe = directFe.reset_index().drop(['index'], axis=1)
            featureRow = pd.concat([featureRow,directFe], axis=1)

        features = features.append(featureRow, ignore_index=True)

    # static cantonal features
    staticCantonal = pd.read_excel("static_data/staticCantonalData.xlsx").set_index('canton').transpose()
    # households
    households = ['1PersonHouseholds', '2PersonHouseholds','3PersonHouseholds', '4PersonHouseholds', 
                  '5PersonHouseholds','6+PersonHouseholds']
    for h in households:
        features[[h+"_perc"]] = staticCantonal.loc[[cantonId]][h][0]/staticCantonal.loc[[cantonId]]['totalHousholds'][0]
    features[['averageHousehold']] = staticCantonal.loc[[cantonId]]['residents'][0]/staticCantonal.loc[[cantonId]]['totalHousholds'][0]
    # add static features
    staticFeatures = ['percentage 65 years or over','urbanPopulationPercent','homeownershipPercent', 
                      'livingSpaceInm2','carsPer1000inhabitants', 'publicTransportationPercent',
                      'privateMotorisedTransportPercent','DoctorsPer100Kinhabitants','residentsPerKm2']
    for f in staticFeatures:
        features[[f]] = staticCantonal.loc[[cantonId]][f][0]
    # construct settlement area feature
    residents = staticCantonal.loc[[cantonId]]['residents'][0]
    settlementArea = staticCantonal.loc[[cantonId]]['areaInKm2'][0]*(staticCantonal.loc[[cantonId]]['settlementAreaPercent'][0]/100)
    features[['residentsPerKm2SettlementArea']] = residents/settlementArea    
    
    if not os.path.exists('data/features'):
        os.makedirs('data/features')
    features.to_csv('data/features/'+cantonId+'.csv', index=False)
    #display(features)

In [195]:
######### constructing the ouputs #########
for cantonId in cantonKeys:
    outputs = pd.DataFrame()
    for t in listOfOutputIntervals:
        outputCols = ['hosp_inz_entries','death_inz_entries','testPositvity','googleMobility']

        outputRow = dict[cantonId][t[0]:t[1]][outputCols].mean().to_frame().transpose()

        '''
        outputRow = pd.DataFrame()
        # direct features
        for o in outputCols:
            directOu = dict[cantonId][t[0]:t[1]][[o]].transpose()
            directOu.columns = [o+'_day_'+str(d) for d in range(0,daysOut)]
            directOu = directOu.reset_index().drop(['index'], axis=1)
            outputRow = pd.concat([outputRow,directOu], axis=1)
        '''    
        outputs = outputs.append(outputRow, ignore_index=True)
    
    if not os.path.exists('data/outputs'):
        os.makedirs('data/outputs')
    outputs.to_csv('data/outputs/'+cantonId+'.csv', index=False)
#display(outputs)




In [196]:
# test cantons are (large, medium, small) (de, fr, de) SG,NE,NW

trainCantons = [canton for canton in cantonKeys if canton not in ['SG','NE','NW']]
# construct train set
train_features = pd.DataFrame()
train_labels = pd.DataFrame()
for cantonId in trainCantons:
    train_features = train_features.append(pd.read_csv("data/features/"+cantonId+".csv"))
    train_labels = train_labels.append(pd.read_csv("data/outputs/"+cantonId+".csv"))

test_features = pd.DataFrame()
test_labels = pd.DataFrame()
# construct test set
for cantonId in ['SG','NE','NW']:
    test_features = test_features.append(pd.read_csv("data/features/"+cantonId+".csv"))
    test_labels = test_labels.append(pd.read_csv("data/outputs/"+cantonId+".csv"))

train_features.to_csv('train_features.csv', index=False)
train_labels.to_csv('train_labels.csv', index=False)
test_features.to_csv('test_features.csv', index=False)
test_labels.to_csv('test_labels.csv', index=False)

In [235]:
# output log file
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn import svm
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
import lightgbm as lightgbm



start = time.time()
train_features = pd.read_csv('train_features.csv')
train_labels = pd.read_csv('train_labels.csv')
test_features = pd.read_csv('test_features.csv')
test_labels = pd.read_csv('test_labels.csv')



pip = Pipeline([('std_scaler', StandardScaler())])
featureNames = train_features.columns
train_features = pip.fit_transform(train_features)
train_labels = pip.fit_transform(train_labels[['hosp_inz_entries']]) #'death_inz_entries','testPositvity','googleMobility'

#linear_model.Ridge(random_state=0),,svm.SVR(),
 #           MLPRegressor(random_state=0),RandomForestRegressor(random_state=0), 
#    , linear_model.Ridge(random_state=0),svm.SVR(),
#, linear_model.Lasso(random_state=0),svm.SVR()

outF = open("output.txt", "w")

for reg in [linear_model.Ridge(random_state=0, alpha=10),linear_model.Lasso(random_state=0, alpha=0.001)]:
    start = time.time()
    display("----------Start evaluating "+str(reg)+"----------")
    scores = cross_val_score(reg, train_features, train_labels, scoring="neg_mean_squared_error", cv=5, n_jobs=-1)
    scores = np.sqrt(-scores)
    display("Scores:"+str(scores))
    #display(scores)
    display("Mean:"+str(scores.mean()))
    #display(scores.mean())
    display("Standard deviation:"+str(scores.std()))
    #display(scores.std())
    display("----------End of evaluating (%s)----------" % (time.time() - start))
'''
reg = linear_model.Lasso(random_state=0, )
reg.fit(train_features, train_labels)
display(pd.DataFrame(reg.coef_.transpose(), index=featureNames, columns=['importance']).sort_values(['importance'], ascending=False))
'''

#lightgbm = lightgbm.LGBMRegressor().fit(pd.DataFrame(train_features),train_labels)
#display(pd.DataFrame(lightgbm.feature_importances_, index=featureNames, columns=['importance']).sort_values(['importance'], ascending=False))

'----------Start evaluating Ridge(alpha=10, random_state=0)----------'

'Scores:[0.37978854 0.42359723 0.33446666 0.44790143 0.28723569]'

'Mean:0.37459790993991093'

'Standard deviation:0.05838485419813887'

'----------End of evaluating (0.23070597648620605)----------'

...

'Scores:[0.34018598 0.32317831 0.27953913 0.33569036 0.24996933]'

'Mean:0.30571262140008565'

'Standard deviation:0.035169570597728184'

'----------End of evaluating (4.8472959995269775)----------'

"\nreg = linear_model.Lasso(random_state=0, )\nreg.fit(train_features, train_labels)\ndisplay(pd.DataFrame(reg.coef_.transpose(), index=featureNames, columns=['importance']).sort_values(['importance'], ascending=False))\n"